In [1]:
import gym
import random
import numpy as np
from keras.models     import Sequential
from keras.layers     import Dense
from keras.optimizers import Adam

Using TensorFlow backend.
/Users/charleskornoelje/Documents/LocalDevelopment/PythonvEnv/344v3.6/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/charleskornoelje/Documents/LocalDevelopment/PythonvEnv/344v3.6/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/charleskornoelje/Documents/LocalDevelopment/PythonvEnv/344v3.6/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be underst

In [12]:
env = gym.make('CartPole-v1')
env.reset()
goal_steps = 500
score_requirement = 60
intial_games = 10000

In [13]:
def play_a_random_game_first():
    for step_index in range(goal_steps):
#         env.render()
        action = env.action_space.sample()
        observation, reward, done, info = env.step(action)
        print("Step {}:".format(step_index))
        print("action: {}".format(action))
        print("observation: {}".format(observation))
        print("reward: {}".format(reward))
        print("done: {}".format(done))
        print("info: {}".format(info))
        if done:
            break
    env.reset()

In [14]:
play_a_random_game_first()

Step 0:
action: 2
observation: [-0.58179651  0.00144494]
reward: -1.0
done: False
info: {}
Step 1:
action: 2
observation: [-0.57891731  0.00287921]
reward: -1.0
done: False
info: {}
Step 2:
action: 2
observation: [-0.57462512  0.00429219]
reward: -1.0
done: False
info: {}
Step 3:
action: 2
observation: [-0.56895172  0.0056734 ]
reward: -1.0
done: False
info: {}
Step 4:
action: 0
observation: [-0.56393922  0.0050125 ]
reward: -1.0
done: False
info: {}
Step 5:
action: 2
observation: [-0.55762491  0.00631431]
reward: -1.0
done: False
info: {}
Step 6:
action: 2
observation: [-0.55005584  0.00756906]
reward: -1.0
done: False
info: {}
Step 7:
action: 0
observation: [-0.54328856  0.00676728]
reward: -1.0
done: False
info: {}
Step 8:
action: 0
observation: [-0.53737369  0.00591487]
reward: -1.0
done: False
info: {}
Step 9:
action: 2
observation: [-0.53035553  0.00701815]
reward: -1.0
done: False
info: {}
Step 10:
action: 1
observation: [-0.52328671  0.00706883]
reward: -1.0
done: False
info: {

In [15]:
def model_data_preparation():
    training_data = []
    accepted_scores = []
    for game_index in range(intial_games):
        score = 0
        game_memory = []
        previous_observation = []
        for step_index in range(goal_steps):
            action = random.randrange(0, 2)
            observation, reward, done, info = env.step(action)
            
            if len(previous_observation) > 0:
                game_memory.append([previous_observation, action])
                
            previous_observation = observation
            score += reward
            if done:
                break
            
        if score >= score_requirement:
            accepted_scores.append(score)
            for data in game_memory:
                if data[1] == 1:
                    output = [0, 1]
                elif data[1] == 0:
                    output = [1, 0]
                training_data.append([data[0], output])
        
        env.reset()

    print(accepted_scores)
    
    return training_data

In [16]:
training_data = model_data_preparation()

[]


In [7]:
def build_model(input_size, output_size):
    model = Sequential()
    model.add(Dense(128, input_dim=input_size, activation='relu'))
    model.add(Dense(52, activation='relu'))
    model.add(Dense(output_size, activation='linear'))
    model.compile(loss='mse', optimizer=Adam())

    return model

In [8]:
def train_model(training_data):
    X = np.array([i[0] for i in training_data]).reshape(-1, len(training_data[0][0]))
    y = np.array([i[1] for i in training_data]).reshape(-1, len(training_data[0][1]))
    model = build_model(input_size=len(X[0]), output_size=len(y[0]))
    
    model.fit(X, y, epochs=10)
    return model

In [9]:
trained_model = train_model(training_data)


Epoch 1/10
10994/10994 [==============================] - 2s 175us/step - loss: 0.2483

In [11]:
scores = []
choices = []
for each_game in range(100):
    score = 0
    prev_obs = []
    for step_index in range(goal_steps):
        # Uncomment below line if you want to see how our bot is playing the game.
        env.render()
        if len(prev_obs)==0:
            action = random.randrange(0,2)
        else:
            action = np.argmax(trained_model.predict(prev_obs.reshape(-1, len(prev_obs)))[0])
        
        choices.append(action)
        new_observation, reward, done, info = env.step(action)
        prev_obs = new_observation
        score+=reward
        if done:
            break

    env.reset()
    scores.append(score)

print(scores)
print('Average Score:',sum(scores)/len(scores))
print('choice 1:{}  choice 0:{}'.format(choices.count(1)/len(choices),choices.count(0)/len(choices)))

KeyboardInterrupt: 